In [8]:
import h5py
import numpy as np
import pandas as pd
from sklearn import preprocessing
from scipy.stats import pearsonr
from torch.utils.data import Dataset
from tqdm import tqdm
import torch
from sklearn.decomposition import PCA

class ECG_Dataset(Dataset):
    def __init__(self, 
                tracings_file_path,
                labels_file_path,
                start = 0,
                end = -1):
        self.f = h5py.File(tracings_file_path, 'r')

        # Get tracings
        self.trace_ids = np.array(self.f['exam_id'])[start:end]
        self.tracings = self.f['tracings']

        # Defining start and end
        self.start = start
        self.end   = (end if end != -1 else len(self.tracings)-1)

        # Get labels
        labels_df = pd.read_csv(labels_file_path)
        self.labels    = {labels_df["exam_id"][i]:labels_df["classe"][i] for i in range(len(self.tracings))}

    def __len__(self):
        return self.end - self.start

    def __getitem__(self, idx):
        # Get tracing
        tracing_idx = self.start + idx
        tracing = np.transpose(self.tracings[tracing_idx])
        
        # Get label
        label = self.labels[self.trace_ids[idx]]

        return tracing, label

In the next cell I try to apply a tucker decomposition, witch is (or should be) a SVD equivalent but for tensors. 

```
from tensorly.decomposition import tucker
import tensorly as tl

with h5py.File('../data/train_dccweek2023.h5', 'r') as f:
    X = f['tracings'][:5000]
    

# aplica o HOSVD mantendo a primeira dimensão
factors = tucker(X, rank=[5000, 2000, 2])

# reconstrói o tensor original usando os componentes principais
X_hosvd = tl.kruskal_to_tensor(factors)

# verifica as dimensões do novo tensor
print(X_hosvd.shape)
```

The above cell uses just way too much memory and takes an eternity to run, and since it is 10x smaller than the full dataset and it needs to be loaded all at once, it cant be a good solution.

The next test should be to iter over each sample individualy, and reduce its dimension using PCA from (1, 4096, 12) to (1, 4096, 2) and build a new training dataset.

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = ECG_Dataset('../data/train_dccweek2023.h5',
                      '../data/train_dccweek2023-labels.csv',
                      start = 0, end = -1)

In [10]:
dataset.tracings

<HDF5 dataset "tracings": shape (51432, 4096, 12), type "<f4">

In [29]:
class ECG_Dataset_Features(ECG_Dataset):
    def __getitem__(self, idx):
        # Get tracing
        tracing_idx = self.start + idx
        tracing = self.tracings[tracing_idx]
        
        # Adding pca of each signal to features
        pca = PCA(n_components=2)
        pca.fit(tracing)
        transformed = np.transpose(pca.transform(tracing))
        
        # Adding fft analysis
        harmonics = []
        for signal in np.transpose(tracing):
            dft = np.fft.fft(signal)
            spectrum = np.abs(dft)
            T = 1/4096
            N = 4096
            f = np.fft.fftfreq(N, T)
            frequencias = f[:N // 2]
            amplitudes = np.abs(dft)[:N // 2] * 1 / N
            best_5 = sorted(zip(amplitudes, frequencias), key = lambda x: x[0])[-1:-6:-1]
            harmonics.append(best_5)
        
        label = self.labels[self.trace_ids[idx]]

        return transformed, harmonics, label

In [30]:
pca_dataset = ECG_Dataset_Features('../data/train_dccweek2023.h5',
                      '../data/train_dccweek2023-labels.csv',
                      start = 0, end = -1)

In [31]:
pca_dataset[0]

(array([[ 0.00132167,  0.00132167,  0.00132167, ...,  0.00132167,
          0.00132167,  0.00132167],
        [-0.00620164, -0.00620164, -0.00620164, ..., -0.00620164,
         -0.00620164, -0.00620164]], dtype=float32),
 [[(0.05926092171471409, 38.0),
   (0.0555799393403267, 9.0),
   (0.05307683713648391, 66.0),
   (0.050414289030029355, 10.0),
   (0.047854997983364136, 28.0)],
  [(0.0367439150385997, 66.0),
   (0.033473448714124755, 28.0),
   (0.031255772613225145, 38.0),
   (0.02823140027741715, 94.0),
   (0.027913665965537095, 75.0)],
  [(0.03552166047944737, 9.0),
   (0.03174460508353783, 10.0),
   (0.029807461527912756, 47.0),
   (0.029384758734274895, 38.0),
   (0.026231576642047612, 75.0)],
  [(0.04507922339990268, 38.0),
   (0.043990256763507866, 66.0),
   (0.040123084905469855, 28.0),
   (0.03835756876954246, 9.0),
   (0.03497336015732579, 10.0)],
  [(0.04534351443579583, 9.0),
   (0.04406721131937605, 38.0),
   (0.040926224167188316, 10.0),
   (0.036991263446432644, 66.0),
 